In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import cv2
import copy
import tqdm
import glob
import keras
import pickle
import logging
import warnings
import numpy as np
import pandas as pd
import tensorflow as tf
from datetime import datetime

In [2]:
warnings.filterwarnings('ignore')
logger = tf.get_logger()
logger.setLevel(logging.ERROR) # or logging.INFO, logging.WARNING, etc.

In [20]:
ROOT_S1 = "_intermediate/stage1_plantdoc"
INT_S1_DIR = sorted(glob.glob(os.path.join(ROOT_S1, "*")))[-1]
print(INT_S1_DIR)
root_s2 = ROOT_S1.replace("stage1", "stage2")

CONTINUE = False

if CONTINUE:
    INT_S2_DIR = sorted([gl for gl in glob.glob(os.path.join(root_s2, "*")) if os.path.isdir(gl)])[-1]
else:
    INT_S2_DIR = f"{root_s2}/{datetime.now().strftime('%Y_%m_%d_%H_%M_%S')}"
print(INT_S2_DIR)

#DATASET_DIR = "_data/plant_pathology"
DATASET_DIR = "_data/plantdoc_csv"

PROB_THRESHOLD = .5

_intermediate/stage1_plantdoc/2024_12_03_08_49_21
_intermediate/stage2_plantdoc/2024_12_06_19_30_57


In [21]:
os.makedirs(INT_S2_DIR, exist_ok=True)

In [22]:
train_data = pd.read_csv(os.path.join(DATASET_DIR, "data.csv"))

In [23]:
def get_latest_checkpoint(dir):
    if not os.path.isdir(dir):
        return None
    ckpts = [file for file in os.listdir(dir) if file.endswith("keras")]
    ckpts.sort()
    return os.path.join(dir, ckpts[-1])

In [24]:
stage1_names = [d for d in os.listdir(INT_S1_DIR) if os.path.isdir(os.path.join(INT_S1_DIR, d)) and d != "patches"]

In [25]:
stage1_names

['SAM + ResNet', 'SAM + YOLOv8', 'SAM + Inception']

In [26]:
stage2_names = ["InceptionV3", "AlexNet", "ResNet152V2"]

In [27]:
def lr(y_true, y_pred):
    return optimizer.learning_rate

In [28]:
def get_lr_metric(optimizer):
    return lr

In [29]:
custom_objects = {
    "lr": lr
}

In [30]:
# Load best performing model by type
stage2_dict = {
    name: keras.models.load_model(f"../disease_detection/out/best{name}_rgb.keras", custom_objects=custom_objects)
    for name in stage2_names
}

In [31]:
import sys
import io
from contextlib import contextmanager

@contextmanager
def filter_output(stdout_condition=None, stderr_condition=None):
    """
    A context manager to filter messages to stdout and stderr.

    :param stdout_condition: A callable that takes a string and returns True if the message should be printed to stdout.
    :param stderr_condition: A callable that takes a string and returns True if the message should be printed to stderr.
    """
    stdout_condition = stdout_condition or (lambda msg: True)
    stderr_condition = stderr_condition or (lambda msg: True)

    class StreamFilter(io.StringIO):
        def __init__(self, condition, original_stream):
            super().__init__()
            self.condition = condition
            self.original_stream = original_stream

        def write(self, msg):
            if self.condition(msg):
                self.original_stream.write(msg)
                self.original_stream.flush()

    original_stdout = sys.stdout
    original_stderr = sys.stderr

    try:
        sys.stdout = StreamFilter(stdout_condition, original_stdout)
        sys.stderr = StreamFilter(stderr_condition, original_stderr)
        yield
    finally:
        sys.stdout = original_stdout
        sys.stderr = original_stderr

In [ ]:
import numpy as np

PAD = False

BATCH_SIZE = 16  # Adjust this value based on your GPU memory and model size
PADDING = 25
# paddings as top right bottom left
PADDINGS = (PADDING, PADDING, PADDING, PADDING)
stage2_results = {}

SKIP_EXISTING = True

with filter_output(lambda x: "gpu_timer" not in x, lambda x: "gpu_timer" not in x):
    for stage2_name, stage2_model in stage2_dict.items():
        for stage1_name in stage1_names:
            stage_name = f'({stage1_name}) + ({stage2_name})'
            print(f'Evaluating {stage_name}')
            stage_dir = os.path.join(INT_S2_DIR, stage_name)
            os.makedirs(stage_dir, exist_ok=True)
            if SKIP_EXISTING and os.path.exists(os.path.join(stage_dir, "data.pkl")):
                print(f"Data for {stage_name} already exists, skipping!")
                continue
            with open(os.path.join(INT_S1_DIR, stage1_name, "data.pkl"), "rb") as file:
                stage1_result = pickle.load(file)
            stage2_result = copy.deepcopy(stage1_result)
            del stage1_result
            #stage2_results[stage_name] = copy.deepcopy(stage1_result)

            # Iterate through the results in batches
            for index, data in tqdm.tqdm(stage2_result.items(), desc=stage_name):
                # Extract the 'healthy' status
                healthy = train_data.iloc[index]['healthy']

                # Collect patches for the current index
                masks = data['masks']
                patches = [mask['patch'] for mask in masks]

                resized_patches = []
                for patch in patches:
                    if PAD and any([padding > 0 for padding in PADDINGS]):
                        padding = tf.constant([[PADDINGS[0], PADDINGS[2]], [PADDINGS[1], PADDINGS[3]], [0, 0]])
                        # Apply padding
                        padded_patch = tf.pad(patch, paddings=padding, mode='CONSTANT', constant_values=0)
                        updated_patch = tf.image.resize(padded_patch, (224, 224))
                    else:
                        updated_patch = tf.image.resize(patch, (224, 224))
                    new_color = tf.constant([.5, .5, .5], dtype=tf.float32)

                    # Create a mask for black pixels
                    black_pixel_mask = tf.reduce_all(updated_patch == [0.0, 0.0, 0.0], axis=-1)  # Shape: (height, width)

                    # Expand mask to match the image dimensions
                    mask_expanded = tf.expand_dims(black_pixel_mask, axis=-1)  # Shape: (height, width, 1)

                    # Replace black pixels with the new color
                    updated_patch = tf.where(mask_expanded, new_color, updated_patch)

                    resized_patches.append(updated_patch)

                batch_inputs = tf.stack(resized_patches)

                # Convert resized patches into a tensor
                #batch_inputs = tf.convert_to_tensor(resized_patches, dtype=tf.float32)

                batch_inputs = tf.expand_dims(batch_inputs, axis=0) if len(batch_inputs.shape) == 3 else batch_inputs

                # Process in mini-batches
                num_patches = len(patches)
                batch_predictions = []
                for start in range(0, num_patches, BATCH_SIZE):
                    end = start + BATCH_SIZE
                    batch = batch_inputs[start:end]
                    predictions = stage2_model(batch).numpy()
                    batch_predictions.extend(predictions)

                # Update the predictions and mask probabilities
                data['predictions'] = []
                for mask, prob in zip(masks, batch_predictions):
                    prob_diseased = prob.squeeze()[1]  # Assuming second index is for 'diseased'
                    data['predictions'].append(prob_diseased)
                    mask['prob_diseased'] = prob_diseased
                    mask['predicted_sick'] = int(prob_diseased > PROB_THRESHOLD)
            with open(os.path.join(stage_dir, "data.pkl"), "wb+") as file:
                pickle.dump(stage2_result, file, protocol=pickle.HIGHEST_PROTOCOL)
            del stage2_result

Evaluating (SAM + ResNet) + (InceptionV3)


(SAM + ResNet) + (InceptionV3):   0%|          | 0/512 [00:00<?, ?it/s]WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
W0000 00:00:1733509907.718431  737925 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1733509907.744574  737925 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1733509907.744972  737925 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1733509907.788923  737925 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1733509907.789339  737925 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1733509907.789745  737925 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1733509907.790131  737925 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0